In [1]:

import numpy as np
import pandas as pd
# import psycopg2

from sklearn.preprocessing import StandardScaler
from sklearn.externals import joblib 
from sklearn.model_selection import train_test_split
# Importing the xgboost
import catboost
from catboost import CatBoostRegressor,Pool
import matplotlib.pyplot as plt
#import matplotlib.pyplot as plt

In [2]:


# ustawienie plików z parametemai modelu
model_path='c:\workspace\Python\polutions\models'
model_season='zima'
model_name='catboost_polution_model'
model_file = model_path+"\\"+model_season+'\\'+model_name+".cmb"
scaler_file=model_path+"\\" +model_name+"\\scaler"
mean_indexbytime_file = model_path+"\\" +model_name+"\\mean_byTime"
board_log =model_path+"\\" +model_name+"\\graph"

In [3]:
# dbuser='global_owner'
# dbpass='if(g=l)weatherpg'
# try:
#     conn = psycopg2.connect(dbname='weatherpg',user=dbuser,host='weatherpg.db.dv.interia.pl',password=dbpass)
# except:
#     print("I am unable to connect to the database")

In [3]:

#dataset_raw =pd.read_sql_query('select * from weather_owner.polution_table_data(119,204)',con=conn)
# dataset_raw =pd.read_sql_query(f'select * from  weather_owner.polution_table_data_season(119, 204,\'{model_season}\')',con=conn)

#dataset_raw =pd.read_sql_query('select * from weather_owner.polution_table_data_current(119,204)',con=conn)
dataset_raw =pd.read_csv('..\\data\\zima\csv\\4970_krakow.csv')
dataset_raw.set_index("data",inplace=True)


In [27]:
#dataset_raw.to_pickle('../picle/polution_raw.pickle')
#dataset_raw=pd.read_pickle('../picle/polution_raw.pickle')
#dataset_raw.read_pickle('../picle/polution_raw.pickle')

# abs(dataset_raw['prev_index_avg']-dataset_raw['index_avg']).mean()
# dataset_raw['index_avg'].std()

23.881761848999197

In [36]:
col_list=['timeofday','is_weekend','windspeed6','dispel50','temp6','prev_index_avg','opady6','rhumid','wind_dir_ns','wind_dir_ew']
col_list_1=['timeofday','is_weekend','windspeed6','temp6','prev_index_avg','opady6','rhumid','dewpt']
col_list_2=['timeofday','is_weekend','dispel50','temp6','prev_index_avg','opady6','rhumid','dewpt']
col_list_2=['timeofday','is_weekend','temp6','prev_index_avg','opady6','rhumid','wind_dir_ns','wind_dir_ew','dewpt']
X =pd.DataFrame(dataset_raw, columns=col_list)
Y =pd.DataFrame(dataset_raw, columns=['index_avg'])
#X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size = 0.2, random_state = 10)

#w naszym przypadku musimy brac całe dni do testowego zbioru bo jak bierzemy pojedyncze godziny to model się teoretycznieie uczy w nieskończoność
#a w praktyce to dopasowuje się do innych godzin tego dnia
uniqe_data=pd.unique(dataset_raw.index)
uniqe_data_train, uniqe_data_test= train_test_split(uniqe_data, test_size = 0.2, random_state = 12)
X_train=X[X.index.isin(uniqe_data_train)]
X_test =X[X.index.isin(uniqe_data_test)]
y_train=Y[Y.index.isin(uniqe_data_train)]
y_test =Y[Y.index.isin(uniqe_data_test)]


In [37]:
cat_features_index = [0,1]

#model = CatBoostRegressor(iterations=10000, learning_rate=0.01, depth=6,l2_leaf_reg=27)

model = CatBoostRegressor(iterations=7000, learning_rate=0.01, depth=6,l2_leaf_reg=27)


In [38]:
#model.fit(X_train, y_train, cat_features_index,                 eval_set=[(X_test, y_test)],use_best_model=True,plot=True,silent=True                 )

#model.fit(X_train, y_train, cat_features_index,    eval_set=[(X_test, y_test)],use_best_model=True,plot=True,silent=True)
model.fit(X_train, y_train, cat_features_index,    eval_set=[(X_test, y_test)],use_best_model=True,silent=True)


In [9]:
model.save_model(model_file, format="cbm")
#model.load_model('polution_catboost.cmb')


In [12]:
model_file

'c:\\workspace\\Python\\polutions\\models\\jesien\\catboost_polution_model.cmb'

In [39]:
score=model.score(X_test, y_test)


In [40]:
pool = Pool(X_train, y_train)

fi = model.get_feature_importance(pool,fstr_type='FeatureImportance')
#finteraction = model.get_feature_importance(pool,fstr_type='Interaction')
feature_importance=pd.DataFrame(fi,columns=['importance'])
feature_importance["futures"]=col_list


feature_importance.set_index("futures",inplace=True)
feature_importance.sort_values(by=["importance"],ascending =False,inplace=True )
feature_importance



,importance
futures,
windspeed6,17.598115
temp6,15.645728
prev_index_avg,15.251470
dispel50,14.023964
timeofday,10.111780
rhumid,7.173036
wind_dir_ns,6.778714
opady6,6.072033
wind_dir_ew,5.517141
